#### package install

In [ ]:
conda install -c anaconda scikit-learn
conda install -c conda-forge jieba
conda install -c conda-forge tensorflow

conda install -c conda-forge keras
conda install -c anaconda gensim

pip install imblearn


import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

#### import packages

In [78]:
import pandas as pd
import numpy as np

import os
import json
os.chdir('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive')
from NLP_function import *
os.chdir('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/Data_Prepare')
#%autosave 60

### pass argv

In [2]:
#file   = sys.argv[1]
#sampid = sys.argv[2]
file = 'C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/Data_Prepare/Project_pogress_20180509.xlsx'

#### Data import _ use original clinical data _ for Unsup_Doc2Vec

In [3]:
sampid = ['A2001','A2002','A2003']

# use original clinical data
df = pd.read_excel(file)
df = df.iloc[:, np.r_[0, 5, 17, 21:58]]
df = df.loc[df['样本编号'].isin(sampid)]

# merge title and content
for a in range(0,df.shape[1]-1):
    if not a in np.r_[0,1,2,3,7:19,22:df.shape[1]]:
        i = df.columns[a]
        df[i] = df[i].apply(lambda x : str(x) if pd.isnull(x) else i.replace('精阅体格检查\n/','') + str(x) )
        df[i] = df[i].apply(lambda x : x.replace('伊阅',''))  
        df[i] = df[i].apply(lambda x : x.replace('nan', ''))  
        #df = df[~df[i].isin([''])]
df.head()

# transform into list
df = df.T
df.columns = df.iloc[0]
res = []
for i in range(0,df.shape[1]):
    res.append(pd.Series(df.iloc[0:,i]).dropna().drop_duplicates().values)
    
# save and export to numpy array
np.save('cli_tokenizing_res.npy', np.array(res))

#### Data import _ training data _ for CN_BOW

In [10]:
df = pd.read_excel('C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/HPO-20200413.xlsx')
df = df.fillna('无')
df['x_cn'] = df['x_cn'].apply(clean_text, lan = 'cn')

In [8]:
# remove str at the begining/end of str
for i in df:
    df[i] = df[i].apply(lambda x: re.sub('\.$', '', x)) 
    df[i] = df[i].apply(lambda x: re.sub('\;$', '', x))
    df[i] = df[i].apply(lambda x: re.sub('^\;', '', x))

df['y_cn'] = df['y_cn'].replace('正常', '无') # change level '正常' to '无'
df['HP'] = df['HP'].replace('正常', '无')     # change level '正常' to '无'
df = df.drop_duplicates(subset='x_cn')       # reset_index(drop=True)  # 4476 unique rows

df.to_excel('C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/cli.tokenizing_all_hpo.xlsx', header=True, index=True) 
df.head()

In [55]:
# change several class (tem training data, will delete after reviewed data)
searchfor = ["自然流产"]
df[df.y_cn.str.contains('|'.join(searchfor))]['y_cn']

639                          复发性自然流产
640                          复发性自然流产
641                          复发性自然流产
642                          复发性自然流产
643                          复发性自然流产
                    ...             
4519           自然流产;胚胎发病;严重的胎儿宫内发育迟缓
4520           自然流产;胚胎发病;严重的胎儿宫内发育迟缓
4521           自然流产;胚胎发病;严重的胎儿宫内发育迟缓
4522    自然流产;胚胎发病;严重的胎儿宫内发育迟缓;减数分裂异常
4523                自然流产;严重的胎儿宫内发育迟缓
Name: y_cn, Length: 138, dtype: object

In [64]:
df[df.y_cn.str.contains('|'.join(searchfor))]

,Unnamed: 0,x_cn,y_cn,hp,comment,confuse,tokenization
639,4015,0-0-3-0,复发性自然流产,HP:0200067,无,NaN,NaN
640,397,反复流产,复发性自然流产,HP:0200067,无,NaN,NaN
641,1940,反复流产相关 反复生化相关基因检查,复发性自然流产,HP:0200067,无,NaN,NaN
642,2917,反复流产相关反复生化相关基因检查+,复发性自然流产,HP:0200067,无,NaN,NaN
643,398,反复胚胎种植失败,复发性自然流产,HP:0200067,无,NaN,NaN
...,...,...,...,...,...,...,...
4519,4411,胎停清宫,自然流产;胚胎发病;严重的胎儿宫内发育迟缓,HP:0005268;HP:0011460;HP:0008846,无,NaN,NaN
4520,5273,体外受精-冷冻胚胎移植后胎停,自然流产;胚胎发病;严重的胎儿宫内发育迟缓,HP:0005268;HP:0011460;HP:0008846,无,NaN,NaN
4521,4199,一次孕8周胎停,自然流产;胚胎发病;严重的胎儿宫内发育迟缓,HP:0005268;HP:0011460;HP:0008846,无,NaN,NaN
4522,4222,2013年结婚 2015年孕1月胎停 流产手术2018年9月第一周期长方案 体外受精 成熟卵...,自然流产;胚胎发病;严重的胎儿宫内发育迟缓;减数分裂异常,HP:0005268;HP:0011460;HP:0008846;HP:0031515,无,NaN,NaN


In [ ]:
# translation via Translator
from translate import Translator
translator = Translator(from_lang='zh', to_lang='en')
df['x_en'] = df['HP']
for p in range(2,len(df)):
    df['x_en'][p] = translator.translate(df['x_cn'][p])

#### Data tranform _ supervised hpo labels

In [15]:
lables_hpo =  pd.read_csv('C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (done)/NLP_for_onedrive/Data_Prepare/Training_data_final.csv',encoding='ANSI')

lables_hpo = pd.DataFrame({'lable': lables_hpo['y_cn'], 'id': lables_hpo['HP']})
lables_hpo = lables_hpo.drop_duplicates().reset_index(drop=True)
lables_hpo['lable1'] = lables_hpo['lable'].apply(lambda x : re.compile('[/; ]').sub('', x))

lables_hpo.to_csv('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/In_Database_cn/lables_hpo.csv', header=True, index=False, encoding="ANSI")

#### Data import _ abbreviation

### import abbreviation

ab = pd.read_excel('C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/abbreviation.xlsx')
ab['leng'] = ab['缩写'].apply(lambda x : len(x))
ab = ab.sort_values(by='leng', ascending=False)
ab = ab[ab.Tag != 'confuse'].sort_values(by='leng', ascending=False)
ab = ab.drop_duplicates(subset='缩写').reset_index(drop=True)
ab.columns = ['abb','zh','en','tag','leng']
ab.to_excel('C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/abbreviation_modified.xlsx', header=True, index=False)

#### Data import _ use other results

In [ ]:
# use postive results data
df = pd.read_csv('clinical_informations_utf8.csv')
df['Clinical_Information.x'] = df['Clinical_Information.x'].str.replace("。", ". ")
df = df.dropna()
df.head(10)

In [3]:
# use project pogres data
df = pd.read_excel('报告阳性结果汇总20200303.xlsx')
df = pd.DataFrame(df,columns=['样本编号','临床信息','项目类型'])
df.columns = ['sample','Clinical_Information.x','Panel_type']
df.head()

,sample,Clinical_Information.x,Panel_type
0,Lib914,无精症。\n第二性征正常。睾丸体积异常，8 mL。输精管正常。精子总数0百万。\n,精阅
1,Lib921,不育。\n第二性征正常。睾丸体积正常。精子总数13百万，正常形态率2%。\n,精阅
2,Lib932,"第二性征正常。睾丸体积正常，17 mL。精子总数0百万。近亲结婚 (父母) , 内脏反位。",精阅
3,Lib957,第二性征正常。睾丸体积异常，4 mL。精子总数0百万。,精阅
4,Lib959,"第二性征正常。睾丸体积正常，12 mL。输精管正常。精子总数0百万。染色体[46,XY]，A...",精阅


In [4]:
df['Clinical_Information.x'] = df['Clinical_Information.x'].str.replace('。', '. ')
df['Clinical_Information.x'] = df['Clinical_Information.x'].str.replace('；', '. ')
df['Clinical_Information.x'] = df['Clinical_Information.x'].str.replace('，', ',')
df.head()
df = df.dropna()
#print(df.Panel_type.value_counts())

#### Data transform used for Hpo_group_data (multi_labled)

In [64]:
Df = pd.read_csv('C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/hpo_import_python/Hpo_group_data/hpo_class_subclass_135_2_unmerged.csv')
Df = Df.replace(np.nan, '', regex=True)
Df['syn_cn'] = Df['syn_cn'].str.replace("No synonyms found for this term.", "")
Df['des_cn'][0]
Df['des_cn'] = Df['des_cn'].apply(lambda x: '' if 'No synonyms ' in x else x)

# remove strange strings
Df['synonyms'] = Df['synonyms'].apply(lambda x : x.replace('No synonyms found for this term.',''))
searchfor = ["No synonyms found for this term",'找不到此术语的同义', '']
length = len(Df[Df.description.str.contains('|'.join(searchfor), na=False)]['description'])
Df[Df.description.str.contains('|'.join(searchfor), na=False)]['description'] = pd.Series(['']*length)

In [65]:
### step_2 expose synonyms and description

df1 = pd.DataFrame({'id'      :   Df['id'].append([Df['id']]*2),
                   'merge'    :   Df['name'].append(Df['synonyms']).append(Df['description']), 
                   'subclass' :   Df['subclass'].append([Df['subclass']]*2)
                   }).reset_index(drop=True)

# df1['merge_new'] = df1['merge'].apply(lambda x : clean_sentence(x, lan='en', med_code=True, abbreviation=False, width=True, symbol=False, character=True))
df1['merge_new'] = df1['merge'].apply(lambda x : clean_text(x, lan='en'))



df1 = df1.drop_duplicates().reset_index()
df1['index'] = df1.index


# split synonyms into multiple rows

df2 = pd.DataFrame({'merge_new_split':df1['merge_new'].str.split(',').explode()})
df2['index'] = df2.index
df2



# merge split data with original data

df = pd.merge(df1, df2, on='index', how='outer')
df = df[['id','merge_new_split','subclass']]
df.rename(columns =  {'merge_new_split': 'merge'}, inplace=True)
df = df.drop_duplicates().reset_index(drop=True)


df.to_excel('C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/hpo_import_python/Hpo_group_data/hpo_subclass_135_3_expode_class_en.xlsx', header=True, index=False)


### step_3 : multiple lables encoder (see R code, will use choix 2 exporting data)
# dense_matrix labels

,new_split,index
0,The second sexual characteristics are normal.,0
0,"The testicular volume is abnormal, 7 mL left, ...",0
0,The vas deferens is normal.,0
0,The total number of sperm is 0 million.,0
0,"Chromosome 46, X, derY.",0
...,...,...
518,Understand the genetic changes of the digestiv...,518
519,9 days after colon cancer surgery.,519
519,"Colon cancer radical surgery on April 1, 2019.",519
519,No family history.,519


In [67]:
### clinical training data transfermation, to merge with hpo_class_25_expode.csv  (code in R ### for gold standard corpus)

df1 = pd.read_csv('C:/Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/Data_Prepare/Training_data_final.csv', encoding='ANSI')
df1 = df1[df1['HP']!='无'].sort_values(by='index').reset_index(drop=True)
df1['index'] = df1.index
df1

df2 = pd.DataFrame({'HP': df1['HP'].str.split(';').explode()})
df2['index'] = df2.index
df2


df = pd.merge(df1, df2, on='index', how='outer')
df = df.drop(['HP_x','index'], axis=1).drop_duplicates().reset_index(drop=True)
df.rename(columns = {'HP_y':'HP'}, inplace = True)
df.to_csv('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/NLP_for_onedrive/Not_In_Database/sup_unsup/Training_data_final_ancenstral_HP.csv', header=True, index=False)

## Sentences tokenization

In [141]:
df1 = pd.read_csv('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/Cluster_R/analysis_output/clinical_informations_all_results.csv', encoding='ANSI')
df1 = df1.dropna(subset=['Clinical_Information.y']).drop_duplicates().reset_index(drop=True)
df1.head()

sentences_x = list()
clix = df['Clinical_Information.y'].tolist()
for i in clix:
    sentences_x.append(nltk.sent_tokenize(i))
        
df1['Clinical_Information_yt'] =  sentences_x
df1['index'] = df1.index

In [146]:
df2 = pd.DataFrame({'new_split': df['Clinical_Information_yt'].explode()})
df2['index'] = df2.index
df2

# merge split data with original data

df = pd.merge(df1, df2, on='index', how='outer')
df = df[['Sample_ID','Panel_Type','Clinical_Information.x', 'new_split']]
df.rename(columns =  {'Clinical_Information.y': 'new_split'}, inplace=True)
df = df.drop_duplicates().reset_index(drop=True)

df.to_csv('C://Users/Jingz/OneDrive/CarrierGene/2019.07.10-HPO (en cours)/Cluster_R/analysis_output/clinical_informations_all_results_token.csv', encoding='ANSI', header=True, index=False)

In [34]:
### sentences tokenization for Chinese version

sentences_x = list()
clix = df['Clinical_Information.y'].tolist()
for i in clix:
    sentences_x.append(nltk.sent_tokenize(i))
        
sentences=[]
for i in sentences_x:
    for b in i:
        sentences.append(b)
sentences = pd .DataFrame(sentences)

print(sentences)

# remove duplication and NA
sentences = sentences.drop_duplicates()
sentences = sentences.dropna()
print('remove duplication and Na, now data dimension is :', sentences.shape)


# export and save
sentences.columns = ['x_cn']
sentences.head()
sentences.to_csv('cli.tokenizing_all.csv', header=True, index=False) 

## Translation

In [8]:
#### translate with python translation module

df = pd.read_csv('cli.tokenizing_all.csv')


#### python translator

from translate import Translator

translator = Translator(from_lang="zh", to_lang="en")
#translation = translator.translate("This is a pen.")
translation = translator.translate(df.iloc[8][0])
translation

# df['x_en'] = df['x_cn'].apply(lambda x:  print(translator.translate(str(x))))
# df.to_csv('cli.tokenizing_all_translate.csv', header=True, index=False) 


#### google translator

#from googletrans import Translator

#translator  = Translator()
#translation = translator.translate("This is a pen.")

'Testis volume is normal, 17 mL.'